In [7]:
# logistic regression
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy  as np
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, MinMaxScaler, OrdinalEncoder
from sklearn.metrics import accuracy_score, make_scorer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import warnings
warnings.filterwarnings("ignore")

In [10]:
num_ftrs = ['minimum_nights', 'number_of_reviews', 'reviews_per_month', 'calculated_host_listings_count', 'time',
           'availability_365', 'latitude', 'longitude']
cat_ftrs = ['neighbourhood_group', 'room_type', 'name', 'race']

In [11]:
# x and y
df = pd.read_csv('preprocessed_for_grid.csv')
label = 'price'
y = df[label]
columns = ['price','neighbourhood']
df.drop(columns, inplace=True, axis=1)
feature_namelist = []
for item in df.columns:
    feature_namelist.append(item)
#print(len(feature_namelist))
feature_names = feature_namelist
label = 'price'
X= df[feature_names]

In [12]:
n_folds = 5
random_state = 2020
def ML_pipeline_kfold_GridSearchCV(X,y,random_state,n_folds,model,param_grid):
    """Use kfold and GridSearchCV to find the best parameter"""
    
    # one-hot encoder
    cat_ftrs = ['neighbourhood_group', 'room_type', 'name', 'race']
    categorical_transformer = Pipeline(steps=[
        ('onehot', OneHotEncoder(sparse=False))])

    # standard scaler
    num_ftrs = ['minimum_nights', 'number_of_reviews', 'reviews_per_month', 'calculated_host_listings_count', 'time',
           'availability_365', 'latitude', 'longitude']
    numeric_transformer = Pipeline(steps=[
        ('scaler', StandardScaler())])
    
    # collect the transformers
    preprocessor = ColumnTransformer(remainder='passthrough', 
        transformers=[
            ('num', numeric_transformer, num_ftrs),
            ('cat', categorical_transformer, cat_ftrs)])
    
    # Setup pipeline
    pipe = Pipeline(steps=[('preprocessor', preprocessor),
                          ('classifier', model)])
    
    # Split train test in a stratified manner
    X_other, X_test, y_other, y_test = train_test_split(X, y, test_size=0.2, random_state = random_state)
    kf = StratifiedKFold(n_splits=n_folds,shuffle=True,random_state=random_state)

    # prepare gridsearch
    grid = GridSearchCV(pipe, param_grid=param_grid,scoring = make_scorer(accuracy_score),
                        cv=kf, return_train_score = True, n_jobs = -1)
    
    grid.fit(X_other, y_other)
    return grid, grid.score(X_test, y_test)

In [ ]:
model = LogisticRegression(penalty = 'l1', solver = 'saga',max_iter = 10000,multi_class='auto')

# tune alpha in l1 for logistic regression
param_grid = {'classifier__C': 1/np.logspace(-4,4,20)}

best_model, best_score = ML_pipeline_kfold_GridSearchCV(X,y,random_state,n_folds,model,param_grid)
c = best_model.best_params_['classifier__C']
results = pd.DataFrame(best_model.cv_results_)
index = np.argmax(results['mean_test_score'])
result_1 = pd.DataFrame(results.iloc[index]['mean_test_score','param_classifier__C', 'mean_train_score'])
display(result_1)
print('Final test score:', best_score)
print('Best C:', c)